figure out how to filter similar nba players by position

try to make good/great amplify the next word

In [1]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from num2words import num2words

In [2]:
ind_to_prospect = {}
prospect_to_ind = {}
prospects = []
with open("prospect_to_docs.json") as f:
    for l in f:
        line = json.loads(l)
        for ind, (prospect, text) in enumerate(line.items()):
            ind_to_prospect[ind] = prospect
            prospect_to_ind[prospect] = ind
            prospects.append(text)
with open("prospect_to_position.json") as f:
    for l in f:
        prospect_to_position = json.loads(l)
        
            
ind_to_player = {}
player_to_ind = {}
players = []
with open("curr_player_to_docs.json") as f:
    for l in f:
        line = json.loads(l)
        for ind, (player, text) in enumerate(line.items()):
            ind_to_player[ind] = player
            player_to_ind[player] = ind
            players.append(text)
with open("curr_player_to_position.json") as f:
    for l in f:
        player_to_position = json.loads(l)

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    stems = []
    for tok in tokens:
        try:
            i = int(tok)
            stems.append(num2words(i))
        except ValueError:
            stems.append(stemmer.stem(tok))
    #stems = [stemmer.stem(token) for token in tokens]
    return stems
tfidf = TfidfVectorizer(stop_words="english", tokenizer=tokenize, ngram_range=(1, 3), norm=None)
docs = tfidf.fit(players+prospects)
prospect_docs = tfidf.transform(prospects).toarray()
player_docs = tfidf.transform(players).toarray()

In [8]:
def find_similarity(query, topk=10):
    transformed = tfidf.transform([query]).toarray().flatten()
    sims = []
    for ind, row in enumerate(prospect_docs):
        doc = row.flatten()
        if not np.all(doc == 0.0):
            dotted = np.dot(doc, transformed)
            sim = dotted/(np.linalg.norm(doc)*np.linalg.norm(transformed))
            sims.append((sim, ind_to_prospect[ind]))
    sorted_sims = sorted(sims, key=lambda x:x[0], reverse=True)
    print query
    for i in range(1, topk + 1):
        out = sorted_sims[i-1]
        print("Rank {}: {}, Similarity: {}".format(i, out[1], out[0]))

In [9]:
find_similarity("good shooter")

good shooter
Rank 1: De Aaron Fox, Similarity: 0.0233413987502
Rank 2: Josh Jackson, Similarity: 0.0191884595667
Rank 3: Tadas Sedekerskis, Similarity: 0.0175138930938
Rank 4: L.J. Peak, Similarity: 0.0168032821348
Rank 5: Kostja Mushidi, Similarity: 0.0152005432887
Rank 6: Jayson Tatum, Similarity: 0.013720729488
Rank 7: Sviatoslav Mykhailiuk, Similarity: 0.0123916783659
Rank 8: Joel Berry, Similarity: 0.012224693362
Rank 9: Markelle Fultz, Similarity: 0.0121713798878
Rank 10: Luke Kennard, Similarity: 0.0120992872442


In [10]:
def find_similar_players(prospect, topk=10):
    if prospect not in prospect_to_ind:
        print "Not a valid prospect"
        return
    prospect_position = prospect_to_position[prospect]
    prospect_ind = prospect_to_ind[prospect]
    prospect_doc = prospect_docs[prospect_ind]
    sims = []
    for ind, row in enumerate(player_docs):
        name = ind_to_player[ind]
        position = player_to_position[name]
        if position in prospect_position:
            doc = row.flatten()
            if not np.all(doc == 0.0):
                dotted = np.dot(doc, prospect_doc)
                sim = dotted/(np.linalg.norm(doc)*np.linalg.norm(prospect_doc))
                sims.append((sim, name))
    sorted_sims = sorted(sims, key=lambda x:x[0], reverse=True)
    print prospect
    for i in range(1, topk + 1):
        out = sorted_sims[i-1]
        print("Rank {}: {}, Similarity: {}".format(i, out[1], out[0]))

In [11]:
find_similar_players("T.J. Leaf")

T.J. Leaf
Rank 1: Kevin Love, Similarity: 0.151849132784
Rank 2: Michael Beasley, Similarity: 0.137175016199
Rank 3: Patrick Patterson, Similarity: 0.134354747327
Rank 4: Terrence Jones, Similarity: 0.132819374028
Rank 5: Lavoy Allen, Similarity: 0.132630150865
Rank 6: Corey Jefferson, Similarity: 0.131583955235
Rank 7: Tyrus Thomas, Similarity: 0.131522765797
Rank 8: Ersan Ilyasova, Similarity: 0.131161698537
Rank 9: Trevor Booker, Similarity: 0.131087866562
Rank 10: Joe Alexander, Similarity: 0.131085314408


In [20]:
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

with open("prospect_to_docs.json") as f:
    for l in f:
        line = json.loads(l)
        text = line["De Aaron Fox"]
        sents = sent_tokenize(text)
        sid = SentimentIntensityAnalyzer()
        for sent in sents:
            if "shoot" in sent:
                print sent
                ss = sid.polarity_scores(sent)
                for k in sorted(ss):
                    print('{0}: {1}, '.format(k, ss[k]))

He does not have an innate sense for how to create high percentage looks for himself at this stage, which is compounded by his struggles as a perimeter shooter, as he often settles for difficult floaters around or outside the paint, which he finds mixed results with.As a shooter, Fox does not have bad mechanics, but struggles to translate that to game settings at the moment.
compound: -0.5065, 
neg: 0.098, 
neu: 0.852, 
pos: 0.05, 
With that said, his struggles as a scorer will make it easier for defenses to game plan against him if he isn't able to improve as a finisher and shooter, particularly in late clock situations.
compound: -0.1043, 
neg: 0.128, 
neu: 0.755, 
pos: 0.117, 
Not a good shooter at all.
compound: 0.4404, 
neg: 0.0, 
neu: 0.58, 
pos: 0.42, 
